In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
os.getcwd()

'd:\\Projects\\breeze_bt'

In [23]:
data = pd.read_csv(f"{os.getcwd()}\\data\\CME_MINI_MNQ1!, 60_f97bf.csv")
data.columns = [c.strip() for c in data.columns]

for c in ['open','high','low','close','DailyPLDot','DailyETop']:
    if pd.api.types.is_numeric_dtype(data[c]):
        continue
    try:
        # clean_vals = [x.strip().replace(',','') for x in data[c]]
        data[c] = data[c].str.replace(',','').str.strip()
        data[c] = pd.to_numeric(data[c], errors='raise')
        data[c] = data[c].round(2)
    except:
        continue

# Convert date columns
data['utc'] = pd.to_datetime(data['utc'])
data['pdt'] = pd.to_datetime(data['pdt'])

C:\Users\WINSTON-ITX\AppData\Local\Temp\ipykernel_16352\2235460076.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['utc'] = pd.to_datetime(data['utc'])
C:\Users\WINSTON-ITX\AppData\Local\Temp\ipykernel_16352\2235460076.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['pdt'] = pd.to_datetime(data['pdt'])


In [61]:
all_plds = data.groupby(data['pdt'].dt.date)['DailyPLDot'].head(1).diff()

In [62]:
all_plds

0           NaN
14          NaN
28      -197.81
37         0.00
60      -229.33
          ...  
12040      0.00
12063    189.77
12086    192.34
12109     75.28
12132     21.38
Name: DailyPLDot, Length: 639, dtype: float64

In [57]:
data['PLD'].isnull().sum()

0.9474724189033427

In [24]:
data.head()

,time,utc,pdt,open,high,low,close,DailyPLDot,DailyETop,DailyXTop,...,DailyEBot,DailyXBot,DailyXBot2,HourlyPLDot,HourlyETop,HourlyXTop,HourlyXTop2,HourlyEBot,HourlyXBot,HourlyXBot2
0,1650556800,2022-04-21 16:00:00,2022-04-21 09:00:00,15165.25,15186.50,15105.75,15112.50,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1650560400,2022-04-21 17:00:00,2022-04-21 10:00:00,15112.50,15151.25,15053.25,15064.50,15220.67,15402.83,15585.0,...,15061.0,14901.33,14741.67,15226.47,15274.11,15321.75,15369.39,15143.11,15059.75,14976.39
2,1650564000,2022-04-21 18:00:00,2022-04-21 11:00:00,15063.75,15063.75,14942.00,14959.25,15220.67,15402.83,15585.0,...,15061.0,14901.33,14741.67,15147.44,15187.39,15227.33,15267.28,15073.97,15000.50,14927.03
3,1650567600,2022-04-21 19:00:00,2022-04-21 12:00:00,14958.75,14971.75,14880.25,14921.25,15220.67,15402.83,15585.0,...,15061.0,14901.33,14741.67,15070.97,15108.28,15145.58,15182.89,15008.11,14945.25,14882.39
4,1650571200,2022-04-21 20:00:00,2022-04-21 13:00:00,14921.25,14936.50,14867.25,14915.50,15220.67,15402.83,15585.0,...,15061.0,14901.33,14741.67,15000.81,15043.11,15085.42,15127.72,14939.36,14877.92,14816.47


In [25]:
data.dtypes

time                    int64
utc            datetime64[ns]
pdt            datetime64[ns]
open                  float64
high                  float64
low                   float64
close                 float64
DailyPLDot            float64
DailyETop             float64
DailyXTop             float64
DailyXTop2            float64
DailyEBot             float64
DailyXBot             float64
DailyXBot2            float64
HourlyPLDot           float64
HourlyETop            float64
HourlyXTop            float64
HourlyXTop2           float64
HourlyEBot            float64
HourlyXBot            float64
HourlyXBot2           float64
dtype: object

In [47]:
# Get Hourly XBot Crosses
data.loc[:,'XBotCrossunder'] = False
data.loc[
    (data['close'] <= data['HourlyXBot']) &
    (data['close'].shift(1) > data['HourlyXBot'].shift(1))
    ,'XBotCrossunder'] = True

data.loc[:,'XTopCrossover'] = False
data.loc[
    (data['close'] >= data['HourlyXTop']) &
    (data['close'].shift(1) < data['HourlyXTop'].shift(1))
    ,'XTopCrossover'] = True

In [48]:
len(data.query('XBotCrossunder == True')) / len(data)

0.05878478511444097

In [40]:
# What happens after XBotCrossunder == True
# Isolate the days where XBotCrossunder == True
true_times = data.query('XBotCrossunder == True')['pdt']
true_days = np.unique([t.date() for t in true_times])

In [50]:
true_times

2       2022-04-21 11:00:00
7       2022-04-21 17:00:00
21      2022-04-22 07:00:00
31      2022-04-24 18:00:00
36      2022-04-24 23:00:00
                ...        
12090   2024-05-08 04:00:00
12106   2024-05-08 21:00:00
12108   2024-05-08 23:00:00
12137   2024-05-10 05:00:00
12139   2024-05-10 07:00:00
Name: pdt, Length: 855, dtype: datetime64[ns]

In [46]:
for time in true_times[0:1]:
    df_filtered = data.loc[(data['pdt'] >= time)]
    print(df_filtered.head())

         time                 utc                 pdt      open      high  \
2  1650564000 2022-04-21 18:00:00 2022-04-21 11:00:00  15063.75  15063.75   
3  1650567600 2022-04-21 19:00:00 2022-04-21 12:00:00  14958.75  14971.75   
4  1650571200 2022-04-21 20:00:00 2022-04-21 13:00:00  14921.25  14936.50   
5  1650578400 2022-04-21 22:00:00 2022-04-21 15:00:00  14919.00  14927.50   
6  1650582000 2022-04-21 23:00:00 2022-04-21 16:00:00  14915.50  14922.25   

        low     close  DailyPLDot  DailyETop  DailyXTop  ...  DailyXBot2  \
2  14942.00  14959.25    15220.67   15402.83   15585.00  ...    14741.67   
3  14880.25  14921.25    15220.67   15402.83   15585.00  ...    14741.67   
4  14867.25  14915.50    15220.67   15402.83   15585.00  ...    14741.67   
5  14893.00  14915.75    15217.25   15416.67   15616.08  ...    14447.25   
6  14875.00  14882.00    15217.25   15416.67   15616.08  ...    14447.25   

   HourlyPLDot  HourlyETop  HourlyXTop  HourlyXTop2  HourlyEBot  HourlyXBot  \
2